In [10]:
# 设置种子
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import torch.optim as optim

import random
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [18]:
# 模型使用 conv2D, transformerEncoder, Mean pooling, MLP, sigmoid
# class Model(nn.Module):
#     def __init__(self, init_weights=False):
#         super(Model, self).__init__()
#         self.conv = nn.Sequential(
#             nn.Conv2d(1, 32, kernel_size=(1, 8)),
#             nn.ReLU(inplace=True),
#         )
#         self.encoder_layer = nn.TransformerEncoderLayer(d_model=128, nhead=2)
#         self.classifier = nn.Sequential(
#             nn.Linear(1024, 128),
#             nn.ReLU(inplace=True),
#             nn.Linear(128, 32),
#             nn.ReLU(inplace=True),
#             nn.Linear(32, 1),
#             nn.Sigmoid(),
#         )
#
#
#     def forward(self, x):
#         x = x.view(-1, 128, 8)
#         x = x.unsqueeze(1)
#         x = self.conv(x)
#         x = x.squeeze(3)
#         x = self.encoder_layer(x)
#         x = torch.mean(x, dim=1)
#         x = self.classifier(x)
#         return x
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(1024, 3000)
        self.fc2 = torch.nn.Linear(3000, 1024)
        self.fc3 = torch.nn.Linear(1024, 512)
        self.fc4 = torch.nn.Linear(512, 1)

        self.bn1 = torch.nn.BatchNorm1d(3000)
        self.bn2 = torch.nn.BatchNorm1d(1024)
        self.bn3 = torch.nn.BatchNorm1d(512)

        self.sigmoid = nn.Sigmoid()
    def forward(self, x):

        out = self.fc1(x)
        out = torch.nn.Dropout(0.3)(out)
        out = torch.nn.ReLU()(out)
        out = self.bn1(out)

        out = self.fc2(out)
        out = torch.nn.Dropout(0.3)(out)
        out = torch.nn.ReLU()(out)
        out = self.bn2(out)

        out = self.fc3(out)
        out = torch.nn.Dropout(0.3)(out)
        out = torch.nn.ReLU()(out)
        out = self.bn3(out)

        out = self.fc4(out)
        out = self.sigmoid(out)
        return out

In [19]:
# 定义超参
class argparse():
    pass

args = argparse()
args.epochs, args.learning_rate, args.train_batch_size, args.test_batch_size = [100, 0.00001, 1024, 512]
args.device, = [torch.device("cuda:0" if torch.cuda.is_available() else "cpu")]

In [20]:
class Dataset_CAR(Dataset):
    def __init__(self, flag='train', csv_paths = []):
        assert flag in ['train', 'test'] # flag 必须是train  test 之间的其中一个
        self.flag = flag
        self.__load_data__(csv_paths)

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.y)

    def __load_data__(self, csv_paths: list):
        # 读取 排列按照 train feature train label test feature test label
        self.x = torch.tensor(pd.read_csv(csv_paths[0]).values)
        self.y = torch.tensor(pd.read_csv(csv_paths[1], header = None).values) # 因为 label的表头是没有的，所以使用 header  = None
        print("feature shape: {}, label shape: {}".format(self.x.shape, self.y.shape))

In [21]:
csv_path_train = ['after_aug_X.csv', 'after_aug_y.csv']
train_dataset = Dataset_CAR(flag='train', csv_paths=csv_path_train)
train_dataloader = DataLoader(dataset=train_dataset, batch_size = args.train_batch_size, shuffle=True)

feature shape: torch.Size([8034, 1024]), label shape: torch.Size([8034, 1])


In [22]:
csv_path_test = ["K_test_feature.csv", "K_test_y.csv"]
test_dataset = Dataset_CAR(flag='test', csv_paths=csv_path_test)
test_dataloader = DataLoader(dataset=test_dataset, batch_size = args.test_batch_size, shuffle=True)

feature shape: torch.Size([7556, 1024]), label shape: torch.Size([7556, 1])


In [23]:
model = Model().to(args.device)
optimizer = optim.Adam(model.parameters(), lr = args.learning_rate)
# 定义二分类交叉熵损失函数
criterion = nn.BCELoss()

train_loss = []
test_loss = []
train_epochs_loss = []
test_epochs_loss = []

In [24]:
# 开始训练
for epoch in range(args.epochs):
    model.train()
    train_epoch_loss = []
    for idx, (data_x, data_y) in enumerate(train_dataloader, 0):
        data_x = data_x.to(torch.float32).to(args.device)
        data_y = data_y.to(torch.float32).to(args.device)
        outputs = model(data_x)
        print(outputs) # output 输出的内容为 nan
        loss = criterion(data_y, outputs)
        loss.backward()
        optimizer.step()

        train_epoch_loss.append(loss.item())
        train_loss.append(loss.item())

        if idx %(len(train_dataloader)//2)==0:
            print("epoch={}/{},{}/{}of train, loss={}".format(epoch, args.epochs, idx, len(train_dataloader),loss.item()))
    train_epochs_loss.append(np.average(train_epoch_loss))

    #=====================valid============================

    model.eval()
    test_epoch_loss = []
    for idx, (data_x, data_y) in enumerate(test_dataloader, 0):
        data_x = data_x.to(torch.float32).to(args.device)
        data_y = data_y.to(torch.float32).to(args.device)

        outputs = model(data_x)

        loss = criterion(data_y, outputs)
        test_epoch_loss.append(loss.item())
        test_loss.append(loss.item())

    test_epochs_loss.append(np.average(test_epoch_loss))

tensor([[0.4537],
        [0.3266],
        [0.4951],
        ...,
        [0.5020],
        [0.4259],
        [0.6351]], grad_fn=<SigmoidBackward0>)
epoch=0/100,0/8of train, loss=50.4622688293457
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], grad_fn=<SigmoidBackward0>)
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], grad_fn=<SigmoidBackward0>)
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], grad_fn=<SigmoidBackward0>)
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], grad_fn=<SigmoidBackward0>)
epoch=0/100,4/8of train, loss=nan
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], grad_fn=<SigmoidBackward0>)
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], grad_

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(121)
plt.plot(train_loss[:])
plt.title("train_loss")
plt.subplot(122)
plt.plot(train_epochs_loss[1:],'-o',label="train_loss")
plt.plot(test_epochs_loss[1:],'-o',label="test_loss")
plt.title("epochs_loss")
plt.legend()
plt.show()

In [25]:
test_true_label = torch.rand(10, 1)
